In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
crawford_emnist_path = kagglehub.dataset_download('crawford/emnist')

print('Data source import complete.')


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
ashishjangra27_face_mask_12k_images_dataset_path = kagglehub.dataset_download('ashishjangra27/face-mask-12k-images-dataset')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Conv2D,MaxPooling2D,ZeroPadding2D,Dropout
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print(tf.__version__)

In [ ]:
data_dir = ashishjangra27_face_mask_12k_images_dataset_path
train_dir = os.path.join(data_dir, "Face Mask Dataset/Train")
test_dir = os.path.join(data_dir, "Face Mask Dataset/Test")
validation_dir = os.path.join(data_dir, "Face Mask Dataset/Validation")

In [ ]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [ ]:
# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)
val_data = val_test_datagen.flow_from_directory(
    validation_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)
test_data = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization

model = keras.Sequential()
model.add(Conv2D(32, (3, 3),input_shape=(128,128,3),activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3),activation="relu"))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(128, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1,activation="sigmoid"))

model.summary()

In [ ]:
from tensorflow.keras.losses import CategoricalCrossentropy
model.compile(loss="binary_crossentropy",optimizer="Adam", metrics=['accuracy'])

In [ ]:
history = model.fit(train_data,validation_data=val_data,epochs=10)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
# Evaluate on test data
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.2f}")

# Predict on test set
preds = model.predict(test_data)
pred_labels = (preds > 0.5).astype("int32")

# Show some predictions
class_names = list(test_data.class_indices.keys())
images, labels = next(test_data)

plt.figure(figsize=(12, 6))
for i in range(6):
    plt.subplot(2, 3, i+1)
    plt.imshow(images[i])
    true_label = class_names[int(labels[i])]
    pred_label = class_names[int(pred_labels[i])]
    color = 'green' if true_label == pred_label else 'red'
    plt.title(f"True: {true_label}\nPred: {pred_label}", color=color)
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

class DummyModel:
    def predict(self, img):
        return np.array([[0.3]])  # Simulated "With Mask"
model1 = DummyModel()

In [ ]:
import gradio as gr
from PIL import Image,ImageOps
"""def crop_center(image, cropx, cropy):
    y, x, _ = image.shape
    startx = x // 2 - cropx // 2
    starty = y // 2 - cropy // 2
    return image[starty:starty + cropy, startx:startx + cropx]"""


def predict_digit(frame):
    try:
        # Convert to grayscale for detection
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

        # Detect faces
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        if len(faces) == 0:
            return "No face detected."

        x, y, w, h = faces[0]  # First face only
        face = frame[y:y+h, x:x+w]  # Crop face
        # Resize the image using PIL before converting to NumPy array

        #frame = crop_center(frame, 20, 20)
        image = Image.fromarray(face.astype('uint8'), 'RGB').resize((128, 128))
        image = np.array(image) / 255.0
        image = image.reshape(1, 128, 128, 3)

        prediction = model.predict(image)
        #pred = prediction[0]

        label = "With Mask 😷" if prediction < 0.5 else "Without Mask 😐"
        confidence = (1 - prediction) if prediction < 0.5 else prediction

        return f"{label}\nConfidence: {np.max(prediction)*100:0.1f}"
    except Exception as e:
      return f"Error: {e}"
demo = gr.Interface(
    fn=predict_digit,
    inputs=gr.Image(),
    outputs="text",
    title="Digit Classifier",
    description="Draw a digit (0–9) and let the model predict it!"
)
demo.launch(share=True)